In [ ]:
from pathlib import Path
import pandas as pd
import os, json, glob, time, subprocess, pathlib
start = 0
SLEEP_BETWEEN = 0

AUTH_USER = os.getenv("AUTH_USER", "samuelzhu")
AUTH_PASS = os.getenv("AUTH_PASS", "Xdb73008762")

TOKEN_URL = "https://analysis.yamaguchi.lan/AppleStockChecker/auth/token/"
TOKEN_TIMEOUT = 90
TOKEN_REFRESH_INTERVAL = 25 * 60  # JWT 有效期 30 分钟，提前 5 分钟刷新
CERT_BUNDLE = "../caddy-ca-bundle.crt"

URL = "https://analysis.yamaguchi.lan/AppleStockChecker/purchasing-price-records/import-tradein-xlsx/?dry_run=0&upsert=1"

RETRIES = 1
BACKOFF = 2.0
TMP_DIR = Path(".tmp_csv_uploads")                # 转换后的临时 CSV 存放目录
TMP_DIR.mkdir(exist_ok=True)
PATTERNS = ["**/*.xlsx", "**/*.xls", "**/*.xlsm", "**/*.xlsb", "**/*.ods", "**/*.csv"]
ENGINE_HINT = {
    ".xlsx": "openpyxl",
    ".xlsm": "openpyxl",
    ".xls":  "xlrd",    # 需要 xlrd<2.0
    ".xlsb": "pyxlsb",
    ".ods":  "odf",
}


def fetch_access_token(token_url: str = TOKEN_URL) -> str:
    """获取 JWT 访问令牌（使用 curl 实现）"""
    auth_bearer = os.getenv("AUTH_BEARER", "").strip()

    headers = ["Accept: application/json"]
    payload = None

    if auth_bearer:
        headers.append(f"Authorization: Bearer {auth_bearer}")
        payload = {}
    elif AUTH_USER and AUTH_PASS:
        payload = {"username": AUTH_USER, "password": AUTH_PASS}
    else:
        raise RuntimeError(
            "缺少获取 token 所需的凭据。请设置以下之一：\n"
            "  1) export ACCESS=...      (直接跳过 token 接口)\n"
            "  2) export AUTH_BEARER=... (用 Bearer token 换取 access token)\n"
            "  3) export AUTH_USER=... && export AUTH_PASS=...\n"
        )

    curl_command = [
        "curl", "--cacert", CERT_BUNDLE,
        "-X", "POST", "--silent", "--show-error", "--fail",
        "--max-time", str(TOKEN_TIMEOUT),
        "-w", "\n%{http_code}",
    ]
    for header in headers:
        curl_command.extend(["-H", header])
    if payload is not None:
        curl_command.extend(["-H", "Content-Type: application/json", "-d", json.dumps(payload)])
    curl_command.append(token_url)

    try:
        result = subprocess.run(
            curl_command, capture_output=True, text=True,
            timeout=TOKEN_TIMEOUT + 5, check=False,
        )
        lines = result.stdout.strip().split("\n")
        response_body = "\n".join(lines[:-1]) if len(lines) >= 2 else result.stdout.strip()
        http_code = lines[-1] if len(lines) >= 2 else "000"

        if result.returncode != 0:
            raise RuntimeError(
                f"请求 token 接口失败：{token_url}\n"
                f"curl 返回码: {result.returncode}\n"
                f"错误: {result.stderr.strip() or 'curl命令执行失败'}\n"
                f"响应: {response_body[:500]}"
            )

        status_code = int(http_code) if http_code.isdigit() else 0
        if not (200 <= status_code < 300):
            raise RuntimeError(
                f"获取 token 失败：HTTP {status_code}\nURL: {token_url}\n响应: {response_body[:800]}"
            )

        try:
            data = json.loads(response_body)
        except json.JSONDecodeError:
            token = response_body.strip()
            if token:
                return token
            raise RuntimeError(f"token 接口返回非 JSON 且为空：{response_body[:200]}")

        for k in ("access", "token", "access_token", "key"):
            v = data.get(k)
            if isinstance(v, str) and v.strip():
                return v.strip()

        raise RuntimeError(
            f"token 接口返回 JSON 但未找到 token 字段：\nkeys={list(data.keys())}"
        )

    except subprocess.TimeoutExpired:
        raise RuntimeError(f"请求 token 接口超时：{token_url} ({TOKEN_TIMEOUT}秒)")
    except FileNotFoundError:
        raise RuntimeError("curl 命令未找到，请确保系统已安装 curl")
    except Exception as e:
        if isinstance(e, RuntimeError):
            raise
        raise RuntimeError(f"请求 token 接口失败：{token_url} ; err={e}") from e


def resolve_access_token() -> str:
    """优先使用环境变量 ACCESS（直接跳过 token 接口）；否则自动获取。"""
    direct = os.getenv("ACCESS", "").strip()
    if direct:
        return direct
    return fetch_access_token()


def list_files(root: str | Path) -> list[str]:
    files: list[str] = []
    for pat in PATTERNS:
        files += glob.glob(str(Path(root) / pat), recursive=True)
    # 过滤临时文件 ~$
    return sorted(f for f in files if not Path(f).name.startswith("~$"))


def to_csv_if_needed(path: str | Path) -> Path:
    """若是 Excel -> 转 CSV 到 TMP_DIR；若已是 CSV 直接返回原路径。"""
    p = Path(path)
    suf = p.suffix.lower()
    if suf == ".csv":
        return p

    engine = ENGINE_HINT.get(suf)
    try:
        df = pd.read_excel(p, engine=engine) if engine else pd.read_excel(p)
    except ImportError as e:
        raise RuntimeError(f"缺少读取 {suf} 的依赖（{e}）。请按需安装：openpyxl / xlrd<2 / pyxlsb / odfpy") from e
    except Exception as e:
        raise RuntimeError(f"读取 {p.name} 失败：{e}") from e

    out = TMP_DIR / (p.parent.name + ".csv")
    try:
        df.to_csv(out, index=False, encoding="utf-8-sig")
    except Exception as e:
        raise RuntimeError(f"写出临时 CSV 失败：{out.name}，{e}") from e
    return out


def post_one(csv_path: Path, access: str, retries: int = 2, backoff: float = 2.0) -> tuple[str, str]:
    """逐个文件 POST；返回 (状态行, 整个响应字符串)"""
    cmd = [
        "curl", "-sS", "-i", "-X", "POST", URL, "--cacert", CERT_BUNDLE,
        "-H", f"Authorization: Bearer {access}",
        "-F", f"files=@{str(csv_path)}",
    ]
    print(f"POST {csv_path}")
    last_err = ""
    for attempt in range(retries + 1):
        try:
            out = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
            text = out.decode("utf-8", "replace")
            status_line = text.splitlines()[0] if text else ""
            return status_line, text
        except subprocess.CalledProcessError as e:
            last_err = e.output.decode("utf-8", "replace")
            if attempt >= retries:
                return f"ERROR after {retries} retries", last_err
            time.sleep(backoff * (attempt + 1))
    return "ERROR", last_err


def main(ROOT):
    files = list_files(ROOT)
    print(f"Found {len(files)} files under {ROOT}")
    files = files[start:]
    success = fail = 0

    # 启动前获取一次 token，循环内按有效期自动刷新
    try:
        ACCESS = resolve_access_token()
        token_fetched_at = time.time()
    except Exception as e:
        raise SystemExit(f"无法获得 ACCESS：{e}")

    for i, f in enumerate(files, 1):
        # JWT access token 有效期 30 分钟，每 25 分钟刷新一次
        if time.time() - token_fetched_at > TOKEN_REFRESH_INTERVAL:
            try:
                ACCESS = resolve_access_token()
                token_fetched_at = time.time()
                print("  [token 已刷新]")
            except Exception as e:
                raise SystemExit(f"刷新 token 失败：{e}")

        src = Path(f)
        source_name = src.parent.name
        print(f"[{i}/{len(files)}] {src}  ->  source={source_name}")

        try:
            csv_path = to_csv_if_needed(src)
        except Exception as e:
            fail += 1
            print(f"  ✗ 转换失败：{e}")
            continue

        status, resp = post_one(csv_path, access=ACCESS, retries=RETRIES, backoff=BACKOFF)
        print("  ", status)
        ok = (status.startswith("HTTP/1.1 202") or status.startswith("HTTP/2 202") or
              status.startswith("HTTP/1.1 200") or status.startswith("HTTP/2 200"))
        if not ok:
            fail += 1
            print("\n".join(resp.splitlines()[:15]))
        else:
            success += 1
        time.sleep(SLEEP_BETWEEN)

    print(f"\nDone. success={success}, fail={fail}, tmp_csv_dir={TMP_DIR.resolve()}")
    # 如需清理临时 CSV，解除注释：
    # shutil.rmtree(TMP_DIR, ignore_errors=True)


ROOT_list = [
    "../shop-data/shop4/",
]

for root in ROOT_list:
    main(root)
    time.sleep(60)